In [1]:
pip install reportlab

Note: you may need to restart the kernel to use updated packages.


In [4]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.platypus import SimpleDocTemplate, PageBreak
import csv
import os

from PIL import Image

def is_valid_image(path):
    try:
        img = Image.open(path)
        img.verify()
        return True
    except (IOError, SyntaxError):
        return False

def main(template_image, csv_file, photo_directory, output_file):
    # Creating a PDF document
    doc = SimpleDocTemplate(output_file, pagesize=letter)
    
    # Opening the PDF document
    c = canvas.Canvas(output_file, pagesize=letter)
    
    # Open the CSV file
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        
        # Loop through each record in the CSV
        for row in reader:
            photo_path = os.path.join(photo_directory, row['photo'])
            if is_valid_image(photo_path):
                data = {'name': row['name'], 'position': row['position'], 'photo': photo_path}
                
                # Generate ID card for the record
                create_id_card(c, template_image, data)
                
                # Add page break for the next ID card
                c.showPage()
    
    # Save the PDF document
    c.save()

def create_id_card(canvas, template_image, data):
    # Adding template image
    img = ImageReader(template_image)
    img_width, img_height = img.getSize()
    
    # Adjusting width and height for the template image
    target_width = letter[0] * 0.8 
    scale_factor = target_width / img_width
    target_height = img_height * scale_factor
    
    # to Calculate position to center the image horizontally and vertically
    x = (letter[0] - target_width) / 2
    y = (letter[1] - target_height) / 2
    
    # Draw template image
    canvas.drawImage(img, x, y, width=target_width, height=target_height)

    # Add employee information
    name_text = f"Name: {data['name']}"
    position_text = f"Position: {data['position']}"
    canvas.setFont("Helvetica", 12)
    canvas.drawString(100, 270, name_text)  # Adjust the Y position for the name
    canvas.drawString(100, 250, position_text)  # Adjust the Y position for the position
    
    # Add employee photo
    employee_photo = ImageReader(data['photo'])
    
    # Adjust the position of the photo
    photo_x = 348  # x axis position of photo
    photo_y = 350  # y axis position of photo
    canvas.drawImage(employee_photo, photo_x, photo_y, width=150, height=150)

if __name__ == "__main__":
    template_image = "ute_id_template.png"
    csv_file = "sampleData.csv" #sample data I considered
    photo_directory = "photos"   # sample photos directory I considered
    output_file = "employee_id_cards.pdf" #the output pdf file will be generated 
    main(template_image, csv_file, photo_directory, output_file)
